This is our **Y_test**, whose size is $3 \times 3$,
$$ 
Y_\text{test} = \begin{bmatrix} 0 & 0 & 0 \\
 0 & 1 &  0\\
  0 & 0& 1 \end{bmatrix}
$$
Columns are different samples. In each column(sample), if $i$ th row is 1, then this sample belongs to $i$ th class. If all 0, then it is in class 0. So, row dimension is $k-1$ , where $k$ is the class number.

For **P_pred**, after my modification, its input should be size $4 \times 3$. Each column still represents each sample. Now, the $i$ th row indicates the probability that this sample is from class $i$. So, in each column, if you sum up all entries, the result is 1. 

In previous function **validation_multi**, I add a line (Please see the code below). This line of code stack an all 1 row vector at the top of original **P_pred**, which is represent the probability of sample classified into class 0.

In [ ]:
#### The first function I modified:
# ! Don't run this code. Just want to show what I have modified.

# In class: SMF_BCD
def validation_multi(self,
                         threshold = 0.5, 
                    result_dict=None,
                    X_test = None,
                    X_test_aux = None,
                    sub_iter=100,
                    verbose=False,
                    stopping_grad_ratio=0.0001,
                    prediction_method_list = ['filter', 'naive', 'alt', 'exhaustive']):
        '''
        Given input X = [data, label] and initial loading dictionary W_ini, find W = [dict, beta] and code H
        by two-block coordinate descent: [dict, beta] --> H, H--> [dict, beta]
        Use Logistic MF model
        '''
        if result_dict is None:
            result_dict = self.result_dict
        if X_test is None:
            X_test = self.X_test
        if X_test_aux is None:
            X_test_aux = self.X_test_aux

        test_X = X_test[0]
        test_Y = X_test[1]

        W = result_dict.get('loading')
        beta = W[1].T
        
        for pred_type in prediction_method_list:
            print('!!! pred_type', pred_type)
            if pred_type == 'filter':
                X0_comp = W[0].T @ X_test[0]
                X0_ext = np.vstack((np.ones(X_test[1].shape[1]), X0_comp))
                if self.d3>0:
                    X0_ext = np.vstack((X0_ext, X_test_aux))
                exp_numerator = np.matmul(W[1], X0_ext)
                exp_numerator = np.exp(exp_numerator)
                normalizer = np.zeros(X_test[0].shape[1])
                for i in range(len(normalizer)):
                    normalizer[i] = 1 + np.sum(exp_numerator[:, i])
                P_pred = np.copy(exp_numerator)

                ################ Only need to add the line below !!!!!!!!!!!!   
                P_pred = np.vstack((np.ones(P_pred.shape[1]),P_pred))    # this line  !!!!!!!!!
                ################ Only need to add the line above !!!!!!!!!!!! 
                  
                for i in range(X_test[0].shape[1]):
                    P_pred[:, i] = P_pred[:, i] / normalizer[i]
                
                accuracy_result = multiclass_accuracy_metrics(Y_test=self.X_test[1], 
                                                            P_pred=P_pred, threshold=threshold)
                if verbose == True:
                    confusion_matrix = accuracy_result.get('confusion_mx')
                    ACC = accuracy_result.get('Accuracy')
                    print('!!! --- Validation --- [confusion_mx, Accuracy] = ', [confusion_matrix, np.round(ACC, 3)])
        return accuracy_result

In [1]:
#### The second function I modified:
# Please run this code to do tests

def multiclass_accuracy_metrics(Y_test, P_pred, result_dict, threshold = 0.5, class_labels=None, use_opt_threshold=False):
    '''
    y_test = multiclass one-hot encoding  labels 
    Q = predicted probability for y_test
    compuate various classification accuracy metrics
    '''

    # k: number of classes, n: number of samples
    # Y_test: (k-1) by n
    # P_Pred: k by n

    Y_test_T = Y_test.T
    P_pred_T = P_pred.T
    results_dict = {}

    count = 0
    for i in np.arange(Y_test_T.shape[0]):
        # Get the predicted class of sample "i" as a number: y1
        y1 = np.arange(P_pred_T.shape[1])[np.max(P_pred_T[i,:]) ==  P_pred_T[i,:]][0]
        
        # Get the true class of sample "i" as a number: y2
        if np.max(Y_test_T[i,:]) == 1:
            y2 = np.sum( np.arange(1,Y_test_T.shape[1]+1) * (Y_test_T[i,:] == 1) )
        else:
            y2 = 0 
        
        if np.abs(y1-y2) < 1e-4: # Compare result
            count = count + 1 # count correctly classified samples
        
    accuracy = count / Y_test_T.shape[0] 

    results_dict.update({'Accuracy': accuracy})
    
    return results_dict

In [2]:
import numpy as np
from sklearn import metrics

Y_test = np.array([[0,0,0],
                   [0,1,0],
                   [0,0,1]])

### Below is our !!! function input P_pred !!!
P_pred = np.array([[0.9 ,0.2 ,0.1],
                   [0.01,0.27,0.2],
                   [0.02,0.23,0.4],
                   [0.07,0.3 ,0.3]])

# If you convert this P_pred into the format of Y_test,
    # Y_pred=
    # 0   0   0
    # 0   0   1
    # 0   1   0
# In this test example, accuracy = 1/3

result_dict = {}
result_dict.update({'Accuracy': np.array([0])})

result_dict = multiclass_accuracy_metrics(Y_test=Y_test, P_pred=P_pred, result_dict=result_dict)

In [3]:
result_dict.get('Accuracy')

0.3333333333333333